In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# --- 1. Define the path to your saved model ---
model_path = None
status = f"Loading model from: {model_path}" if model_path else "No model path found"
print(status)

# --- 2. Load the fine-tuned model and tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# --- 3. Load and prepare the SAME validation data ---
print("Loading and preparing data...")
df = pd.read_csv('./dev/data/sentiment_data.csv', encoding='latin1')
df.dropna(inplace=True)

# Recreate the same label mappings from your training script
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
df['label'] = df['Sentiment'].map(label_map)

# Split the data with the same random_state to get the same validation set
_, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [ ]:
val_encodings = tokenizer(val_df['Sentence'].tolist(), truncation=True, padding=True)

class FinancialDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

val_dataset = FinancialDataset(val_encodings, val_df['label'].tolist())

In [ ]:
print("Running predictions on the validation set...")

# We still use a Trainer, but only to run the prediction
trainer = Trainer(model=model)
predictions_output = trainer.predict(val_dataset)

# The actual predictions are the argmax of the logits
y_pred = np.argmax(predictions_output.predictions, axis=1)
y_true = predictions_output.label_ids

In [ ]:
print("\nGenerating confusion matrix...")
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.config.id2label.values())

disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix for Fine-Tuned FinBERT")
plt.show()

In [ ]:
print(classification_report(y_true, y_pred, target_names=['positive', 'negative', 'neutral']))